## Imports

In [ ]:
# Program Version 2.0

import networkx as nx
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch_geometric.data import Data
from torch_geometric.nn import GCNConv, global_mean_pool
from torch_geometric.utils import from_networkx

c:\Users\Will\miniforge3\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Function Definitions

In [ ]:
# 1. Generate a random graph
def generate_graph(num_nodes=100, edge_prob=0.05):
    G = nx.erdos_renyi_graph(n=num_nodes, p=edge_prob)
    return G

# 2. Compute graph properties
def compute_property(G):
    largest_cc = max(nx.connected_components(G), key=len)
    return len(largest_cc) / G.number_of_nodes()

# 3. Convert to PyTorch Geometric format
def graph_to_pyg_data(G):
    pyg_data = from_networkx(G)
    return pyg_data

## GNN Model

In [ ]:
# 4. Define a simple GNN model
class GNNModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(GNNModel, self).__init__()
        self.conv1 = GCNConv(input_dim, hidden_dim)
        self.conv2 = GCNConv(hidden_dim, hidden_dim)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index # Initial node features
        x = self.conv1(x, edge_index).relu()
        x = self.conv2(x, edge_index).relu()
        x = global_mean_pool(x, data.batch)
        return self.fc(x)


## Training Code

In [4]:
# 5. Training loop
def train_model(model, data, target, epochs=100, lr=0.01):
    optimizer = optim.Adam(model.parameters(), lr=lr)
    loss_fn = nn.MSELoss()
    
    for epoch in range(epochs):
        model.train()
        optimizer.zero_grad()
        output = model(data)
        loss = loss_fn(output, target)
        loss.backward()
        optimizer.step()
        if epoch % 10 == 0:
            print(f"Epoch {epoch}, Loss: {loss.item()}")

## Execution

In [ ]:
# Example Usage
G = generate_graph()
pyg_data = graph_to_pyg_data(G)
pred_property = compute_property(G)

# Add dummy node features
num_nodes = G.number_of_nodes()
pyg_data.x = torch.eye(num_nodes) # One hot encoding for x
pyg_data.batch = torch.zeros(num_nodes, dtype=torch.long)

target = torch.tensor([pred_property], dtype=torch.float32).unsqueeze(1)

model = GNNModel(input_dim=num_nodes, hidden_dim=32, output_dim=1)
train_model(model, pyg_data, target)

Epoch 0, Loss: 0.8589601516723633
Epoch 10, Loss: 0.014687900431454182
Epoch 20, Loss: 0.0237444955855608
Epoch 30, Loss: 0.0003009259235113859
Epoch 40, Loss: 2.8765862225554883e-06
Epoch 50, Loss: 0.0001663745060795918
Epoch 60, Loss: 5.5031687224982306e-05
Epoch 70, Loss: 1.494654770795023e-05
Epoch 80, Loss: 2.512766883455697e-09
Epoch 90, Loss: 3.1333911465480924e-06
